# Import libraries

In [1]:
#!pip install os
#!pip install openai

In [2]:
import os
from openai import OpenAI

# Add API Key and create client variable

In [ ]:
os.environ['API_KEY'] = 'INSERT KEY HERE'
# os.environ['API_KEY'] = ''

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv('API_KEY'),
)

print(client.api_key)

### Get sample responses to check whether it works ok

In [4]:
response = client.chat.completions.create(
    max_tokens=50,
    messages=[
        {
            "role": "user",
            "content": "Once upon a time",
        }
    ],
    model="gpt-3.5-turbo",
)

In [5]:
print(response)
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-8zoaTPOzHeTJplAL053LU9AzRezWO', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content=', in a small village nestled among the mountains, there lived a young girl named Lily. Lily was known throughout the village for her kind heart and adventurous spirit. She spent her days exploring the forests and meadows that surrounded her home, always on the', role='assistant', function_call=None, tool_calls=None))], created=1709743157, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_b9d4cef803', usage=CompletionUsage(completion_tokens=50, prompt_tokens=11, total_tokens=61))
, in a small village nestled among the mountains, there lived a young girl named Lily. Lily was known throughout the village for her kind heart and adventurous spirit. She spent her days exploring the forests and meadows that surrounded her home, always on the


In [6]:
prompt = "What are your opening hours of the zoo?"

In [7]:
response = client.chat.completions.create(
    max_tokens=50,
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo",
)

In [8]:
print(f'Model: {response.model}')
print(f'Created: {response.created}')
print(f'ID: {response.id}')

Model: gpt-3.5-turbo-0125
Created: 1709743210
ID: chatcmpl-8zobK0R78HVj9gGUBEAClzkPRv2ID


In [9]:
print(response.choices[0].message.content.strip())

I apologize for the inconvenience, but as a language model AI, I do not have real-time information. I recommend checking the zoo's official website or contacting them directly to inquire about their opening hours.


# Roles
There are three important roles.
- User. This one is the human user that interacts with the Chatbot.
- System. Sets the behavior of the assistant at the beginning. Sets like a brief for the engine.
- Assistant. This is the role that replies to the prompts. We can also use it to feed info to itself.


In [10]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that speaks in Spanish."
    },
    {
        "role": "user",
        "content": "Who won the last World Cup in Brazil?"
    },
    {
        "role": "assistant",
        "content": "You will always start the conversation with a colloquial Spanish greeting."
    }
]

In [11]:
response = client.chat.completions.create(
    messages=messages,
    model="gpt-3.5-turbo",
)

In [12]:
# print(response)
print(response.choices[0].message.content.strip())

¡Hola! El último Mundial de la FIFA se celebró en Brasil en 2014 y fue ganado por Alemania. ¿Hay algo más en lo que pueda ayudarte?


# Configure assistant for Coca Cola Museum

In [13]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant for a Coca Cola museum. \n"
                    "You have detailed knowledge about the company's history, products, items in the store, entry fee and the opening and closing times of the place. You also have inside knowledge of the museum's service."
    },
    {
        "role": "user",
        "content": "How much is one notebook in the souvenir store? And can I go on Sundays at 5am?"
    }
]

In [14]:
response = client.chat.completions.create(
    messages=messages,
    model="gpt-3.5-turbo",
)

In [15]:
print(response.choices[0].message.content)

Hello! A Coca Cola notebook in the souvenir store typically costs $10. You can visit the museum on Sundays, but please note that the museum opens at 9am. Unfortunately, you won't be able to visit at 5am on Sundays as the museum will still be closed at that time. The opening hours on Sundays are usually from 9am to 5pm. If you have any more questions or need further information, feel free to ask!


### Create context for a fluent conversation

In [16]:
messages.append(
    {
        "role": "assistant",
        "content": response.choices[0].message.content
    }
)

In [17]:
messages.append(
    {
        "role": "user",
        "content": "How about on Monday?"
    }
)

In [18]:
response = client.chat.completions.create(
    messages=messages,
    model="gpt-3.5-turbo",
)

In [19]:
print(response.choices[0].message.content)

The Coca Cola museum is typically open from Monday to Saturday from 9am to 5pm. So, yes, you can visit the museum on a Monday during those hours. If you have any other questions or need more information, please don't hesitate to ask!


### Experiment with temperature
This changes the randomness of the output.

In [20]:
# High temperature
high_response = client.chat.completions.create(
    messages=messages,
    model="gpt-3.5-turbo",
    temperature=1.6,
    max_tokens=100
)

# Low temperature
low_response = client.chat.completions.create(
    messages=messages,
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=100
)

In [21]:
print("High temperature:", high_response.choices[0].message.content)
print("Low temperature:", low_response.choices[0].message.content)

High temperature: On Mondays, the Coca Cola museum is usually open from 9am to 5pm, with the last entry at 4pm. This schedule could be subject to change on holidays or special occasions, so it's always a good idea to confirm the hours beforehand. If you have any more questions or if there's anything else you'd like to know, feel free to ask!
Low temperature: On Mondays, the Coca Cola museum is typically open from 9am to 5pm as well. So, you can plan your visit between those hours on a Monday. If you have any other questions or need more details, feel free to ask!


### Checking and printing errors, like model's max_tokens exceeded
More errors can be found in https://help.openai.com/en/articles/6897213-openai-library-error-types-guidance 

In [24]:
messages.append(
    {
        "role": "user",
        "content": "Tell me everything you know about Coca Cola with a lot of detail."
    }
)

In [26]:
try:
    response = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        temperature=0.6,
        max_tokens=5000
    )
    print(response.choices[0].message.content)
except Exception as e:
    print(f'Got an error from OpenAI API: {e}')

Got an error from OpenAI API: Error code: 400 - {'error': {'message': 'max_tokens is too large: 5000. This model supports at most 4096 completion tokens, whereas you provided 5000.', 'type': 'invalid_request_error', 'param': 'max_tokens', 'code': None}}


In [27]:
try:
    response = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        temperature=0.6,
        max_tokens=256
    )
    print(response.choices[0].message.content)
except Exception as e:
    print(f'Got an error from OpenAI API: {e}')

Coca-Cola is one of the most iconic and recognizable brands in the world, known for its signature red and white logo and refreshing beverages. Here are some key details about Coca-Cola:

1. **History**: Coca-Cola was invented in 1886 by pharmacist John Stith Pemberton in Atlanta, Georgia. The drink was originally intended as a patent medicine, claiming to cure various ailments. However, it soon became popular as a refreshing soda beverage.

2. **Products**: Coca-Cola offers a wide range of beverages, including the classic Coca-Cola soda, Diet Coke, Coca-Cola Zero Sugar, Sprite, Fanta, and many more. The company also produces various non-carbonated drinks like juices, teas, and bottled water.

3. **Global Reach**: Coca-Cola is sold in over 200 countries worldwide, making it one of the most widely distributed products in the world. The brand's marketing campaigns and iconic logo have contributed to its global recognition.

4. **Sustainability**: Coca-Cola is committed to sustainability a